# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

2つのモデルを用いて、入力されたシーケンスに基づいた別のシーケンスを出力するモデル。

これまで、RNN（やLSTM）を用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できるのではないか。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプション生成などに応用できそう。隠れ状態を通じてRNNからCNNまで逆伝播が繋がるので、画像と文章のペアさえ用意すれば学習できそう。というかできる[1]。

[1] [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/abs/1411.4555)

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。

本章ではこの翻訳モデルを作成する。入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。こういった、シーケンスをシーケンスに変換するモデルを *Seq2Seq (Sequence to Sequence)* と呼ぶ。

Seq2Seqは以下の二つのRNNから構成される。

- ***Encoder*** : 文章ベクトルを生成するRNN
- ***Decoder*** : 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。

In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings; warnings.filterwarnings("ignore")
import math
import random

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットから日本語と英語の対訳コーパスを使用する。

- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [3]:
ds = tfds.load(
    "huggingface:iwslt2017/iwslt2017-en-ja",
    data_dir="data",
    split="train"
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample["translation"]["ja"].decode()
    en = sample["translation"]["en"].decode()
    data_ja.append(ja)
    data_en.append(en)

print("num of data:", len(data_ja), "\n")
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

腐敗が 病的に民主主義を 破壊してしまうということです つまり どのようなシステムであっても その構成員がごく少数の 構成員によって選ばれる場合には その構成員がごく少数の 構成員によって選ばれる場合には それは ごく少数の構成員が ごくごく少数の構成員が 改革を阻止できることを意味します
It's a pathological, democracy-destroying corruption, because in any system where the members are dependent upon the tiniest fraction of us for their election, that means the tiniest number of us, the tiniest, tiniest number of us, can block reform.

これは大変なことです
This is a big deal.

疑問というのは 西洋社会におけるすべての議論は 課税レベルに関することです
We ask the question -- the whole debate in the Western world is about the level of taxation.

つまり大きな意味では 技術というのは何も こんな機器だけをさすのではないのです 習慣、テクニック、心理的手法 なども含めて 技術と呼べるのです
So in a broad sense, we don't need to think about technology as only little gadgets, like these things here, but even institutions and techniques, psychological methods and so forth.

ひも理論が難しいなんて人もいますが 楽なもんです
And some people think that string theory is tough.



書き出し

In [5]:
textfile_ja = "data/iwslt2017_ja.txt"
with open(textfile_ja, "w") as f:
    f.write("\n".join(data_ja))

textfile_en = "data/iwslt2017_en.txt"
with open(textfile_en, "w") as f:
    f.write("\n".join(data_en))

読み込み

In [6]:
textfile_ja = f"data/iwslt2017_ja.txt"
textfile_en = f"data/iwslt2017_en.txt"

with open(textfile_ja) as f:
    data_ja = f.read().splitlines()
with open(textfile_en) as f:
    data_en = f.read().splitlines()

### 前処理

日本語、英語別々にトークナイザを作成する。

In [7]:
tokenizer_prefix_ja = f"models/tokenizer_iwslt2017_ja"
tokenizer_prefix_en = f"models/tokenizer_iwslt2017_en"
pad_id = 3
vocab_size = 8000

In [ ]:
spm.SentencePieceTrainer.Train(
    input=textfile_ja,
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size,
    pad_id=pad_id
)

spm.SentencePieceTrainer.Train(
    input=textfile_en,
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size,
    pad_id=pad_id
)

In [9]:
sp_ja = spm.SentencePieceProcessor(f"{tokenizer_prefix_ja}.model")
sp_en = spm.SentencePieceProcessor(f"{tokenizer_prefix_en}.model")

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print("num of vocabrary (ja):", n_vocab_ja)
print("num of vocabrary (en):", n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [10]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [11]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 学習データの作成

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Decoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

`DataLoader`の作成。

In [12]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # Encoderへの入力
        x_dec = en[:-1] # Decoderへの入力
        y_dec = en[1:] # Decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("num of train data:", len(train_dataset))
print("num of test data:", len(test_dataset))

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

# example
x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

num of train data: 178487
num of test data: 44621


(torch.Size([32, 109]), torch.Size([32, 93]), torch.Size([32, 93]))


---

## Seq2Seqを用いた翻訳モデル

Encoder、Decoderを作成し、Seq2Seqモデルを作成する。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN。LSTMと線形層で作る。

ここで、何を隠れ状態とするかを考える。Decoderに渡したい隠れ状態として満たしてほしい条件は以下である。

- （padを除いた）全ての入力を参照して出力されている
- 固定長

これらを全て満たしていれば何でもよい。

よくあるのは最後の隠れ状態。これが一番シンプルで実装も簡単。

ただこれだと学習時にpadトークンが隠れ状態に関与してしまう。多くの場合、推論時にpadトークンは存在しないため、学習時もpadトークンを考慮しない出力をさせた方が良さそう。また、padの数が多くなるにつれて隠れ状態がある一定の値に収束してしまう（経験談）。RNNに同じトークンを何度も入力することで隠れ状態が収束してしまうみたい。そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、入力文を考慮した出力が行えない。

ではどうするかというと、「padを除いた最後の隠れ状態」とする。eosトークンが入力された時点の隠れ状態とも言える。これで上記の問題を解決できる。

他にもいくつかの案が考えられる。例えば、「padを除いた全ての隠れ状態の平均」とか。これも条件を満たす。

また、RNNを双方向にするという手法もある。Encoderは入力文の特徴を抽出できれば良いので、文の初めから順に処理しなければならない訳ではない。例えば逆から処理してもよい。

双方向RNNでは、順方向と逆方向の双方向で演算を行い、各時刻で二つの隠れ状態を出力する。両端の時刻の隠れ状態を取得することで二種類の隠れ状態が得られ、それらを足すないしは結合することで最終的な一つの隠れ状態が得られる。

双方向RNNは`torch.nn.RNN`の引数`bidirectional=True`で実装できる。ただ、各隠れ状態の出力がpadトークンを除いて行われて欲しいという望みがあり、それを実現するための実装が難しそうなので、今回は不採用。

結局どうしようかという感じだが、無難に「padを除いた最後の隠れ状態」を採用する。Encoderを以下のように実装する。

In [13]:
class Encoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_pos = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, _ = self.lstm(x) # (batch_size, seq_len, hidden_size)
        h = hs[eos_pos] # (batch_size, hidden_size)
        h = self.dropout(h)
        h = self.fc(h) # (batch_size, hidden_size)
        return h

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層で作る。

In [14]:
class Decoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hc):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, hc = self.lstm(x, hc) # (batch_size, seq_len, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        return y, hc

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        hc = self.get_hc(h)
        y, _ = self.decoder(x_dec, hc)
        return y

    def get_hc(self, h):
        h = h.unsqueeze(0)
        c = torch.zeros_like(h)
        hc = (h, c)
        return hc

In [16]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 16,761,152



---

## 実践

### 学習

In [17]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model):
    model.eval()
    losses = []
    for x_enc, x_dec, y_dec in test_loader:
        x_enc = x_enc.to(device)
        x_dec = x_dec.to(device)
        y_dec = y_dec.to(device)

        y = model(x_enc, x_dec)
        loss = loss_fn(y, y_dec)
        losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = math.exp(loss)
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [18]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [19]:
train(model, optimizer, n_epochs=20, prog_unit=1)

 1/20: #################### 100% [00:02:19.34] ppl train: 173.17, test: 111.86 
 2/20: #################### 100% [00:02:17.17] ppl train: 97.66, test: 85.91  
 3/20: #################### 100% [00:02:16.70] ppl train: 78.36, test: 73.13 
 4/20: #################### 100% [00:02:16.97] ppl train: 67.43, test: 65.63 
 5/20: #################### 100% [00:02:17.20] ppl train: 60.25, test: 60.59 
 6/20: #################### 100% [00:02:16.97] ppl train: 54.97, test: 57.03 
 7/20: #################### 100% [00:02:17.02] ppl train: 50.86, test: 54.11 
 8/20: #################### 100% [00:02:17.59] ppl train: 47.50, test: 51.97 
 9/20: #################### 100% [00:02:17.06] ppl train: 44.67, test: 50.31 
10/20: #################### 100% [00:02:17.68] ppl train: 42.29, test: 48.80 
11/20: #################### 100% [00:02:17.02] ppl train: 40.26, test: 47.76 
12/20: #################### 100% [00:02:15.11] ppl train: 38.44, test: 46.93 
13/20: #################### 100% [00:02:18.09] ppl train: 36.

In [20]:
model_path = "models/lm_seq2seq.pth"
torch.save(model.state_dict(), model_path)

### 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。決定的な出力にする。

In [21]:
def token_sampling(y, decisive=True):
    y = y.squeeze(0, 1)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    h_enc = model.encoder(in_ids)
    hc = model.get_hc(h_enc)
    next_token = bos_id

    token_ids = []
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).reshape(1, 1)
        y, hc = model.decoder(x, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データから。

In [22]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: お気づきでしょうか どの国でも 同じ3線が急勾配です
output: And you can see that the most important thing is that the average person in the world is not going to be the same as the United States.
answer: Notice also that on all of these graphs, the slope is steeper on in-group, authority, purity.

input: こうして男性職員の割合は 14%から7%に落ち込みました
output: And the average American soldiers were killed.
answer: We went from 14 percent to seven percent.

input: でもホントにインターネットは世界を変えた
output: But I think we're going to be able to do this for the future.
answer: No, but it's true: the Internet has changed the world.

input: 覚えていてください 環境の問題とは 溶けている氷河や凍土の事だけではなく 私たちの子どもの問題でもあるのです
output: And the reason why this is, is that the world is not just a little bit of the  ⁇ -ray of the Earth's body, and it's a very powerful way to get it.
answer: And my urging is that when we think about environmental issues that we remember that it's not just about melting glaciers and ice caps, but it's also about our children as well.

input: 骨が見え

訓練データに含まれていないものも試してみる。

In [23]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: でも私は こんなことを 考えるようになりました 「今までとは違う方法で ストーリーを語るために 現代の進歩した技術を使えないか ― 私たちが この100年使ってきた 旧来の映像制作のツールでは 語り切れないような 別の形のストーリーを語れないか?」
output: But I wanted to be a part of the world, and I'd like to ask you, "How do you think about the world's world, and the world's most important, and the world's most important, and I'm going to do this, and I'm going to do this, and I'm going to do this with the world."
answer: But I started thinking about, is there a way that I can use modern and developing technologies to tell stories in different ways and tell different kinds of stories that maybe I couldn't tell using the traditional tools of filmmaking that we've been using for 100 years?

input: うーん
output: Yeah.
answer: Mmm.

input: あの時の父さんは 砂浜から僕に叫んでたよね」
output: And I'm going to go to the door, and I'm going to go to the door."
answer: I remember my image of you was that you were up on the shore yelling at me."

input: これで年齢の話はもういいわね
output: So I'm going to talk about the future of the future.
a

In [24]:
# original
sentences = [
    "ありがとう",
    "猫はかわいいのです",
    "上手く文章が書けるようになりました"
]

for sentence in sentences:
    print("input:", sentence)
    print("output:", translate(model, sentence))
    print()

input: ありがとう
output: Thank you.

input: 猫はかわいいのです
output: The sympathy is a little bit of a good thing.

input: 上手く文章が書けるようになりました
output: And they were all the way to read.

